# Example notebook for analysing event and cluster data from Krakow Test Beam
+ I give some example histograms with various cuts and stuff
+ Important was to establish the procedure for getting the corryvreckan data and looping over the trees

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1">Import libraries</a></span></li><li><span><a href="#Read-root-file-and-convert-to-python" data-toc-modified-id="Read-root-file-and-convert-to-python-2">Read root file and convert to python</a></span></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-3">Analysis</a></span><ul class="toc-item"><li><span><a href="#Cluster-size-(also-column-width-/-row-width-of-clusters)" data-toc-modified-id="Cluster-size-(also-column-width-/-row-width-of-clusters)-3.1">Cluster size (also column width / row width of clusters)</a></span></li><li><span><a href="#Number-of-clusters" data-toc-modified-id="Number-of-clusters-3.2">Number of clusters</a></span></li><li><span><a href="#Correlation-plots" data-toc-modified-id="Correlation-plots-3.3">Correlation plots</a></span></li></ul></li></ul></div>

## Import libraries

In [1]:
from ipywidgets import IntProgress
from IPython.display import display,HTML,clear_output
import time
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from uits3_krakow22.src.Event import Event
from uits3_krakow22.src.Cluster import Cluster
from uits3_krakow22.src.Track import Track
from uits3_krakow22.src.Utils import *


display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

Welcome to JupyROOT 6.24/06


## Read root file and convert to python
+ Converting to python classes is actually not needed in this example and we could very well just take the corryvreckan cluster objects for filling histograms
+ However in the long run we will probably need to add our own stuff to clusters so I already do the conversion here

In [ ]:
filePath = "/home/cehrich/Software/uits3_krakow22/corry-tools/output/exampleData_clusters.root"
file = ROOT.TFile.Open(filePath)
tEvents = file.Get("Event")
tClusters = file.Get("Cluster")

#tClusters.GetListOfBranches().ls() # list detectors
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]

events = []

nEvents = tClusters.GetEntriesFast() # to read full file
#nEvents = 1000

#progress bar
f = IntProgress(min=0, max=nEvents, description="Converting:")
display(f)

for index,event in enumerate(tClusters):
    if index > nEvents : break #to limit the number of events to be processed
    myEvent = Event()
    for detector in detectors:
        branch = getattr(event, detector)
        for cluster in branch:
            myClus = Cluster()
            myClus.setData(cluster)
            myEvent.addCluster(myClus)
    events.append(myEvent)
    f.value += 1
    
f.bar_style = "success"

IntProgress(value=0, description='Converting:', max=30000)

In [ ]:
filePath = "/home/cehrich/Software/uits3_krakow22/corry-tools/output/exampleData_clusters.root"
file = ROOT.TFile.Open(filePath)
tEvents = file.Get("Event")
tClusters = file.Get("Cluster")

detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]

events = []

nEvents = tClusters.GetEntriesFast()

for index,event in enumerate(tClusters):
    for detector in detectors:
        branch = getattr(event, detector)
        for cluster in branch:
            print(cluster.local)

In [4]:
import pickle
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
with open("/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456114514_221112120005.pkl", 'rb') as f:
    events = pickle.load(f)
    
len(events)

100000

## Analysis

### Cluster size (also column width / row width of clusters)
+ Since we observe that the inner layers have much more hits, we know they must be from delta electrons or from the beam etc.
+ In order to get rid of them, one idea would be to apply cuts on cluster size as the wanted proton hits should lead to larger clusters

In [5]:
%jsroot on
canvas = ROOT.TCanvas()
legend = ROOT.TLegend()

# Calculate histogram
hClusSize = [ROOT.TH1F("Cluster_Size_"+det, "Size of cluster - "+det, 100, 0, 100) for det in detectors]
for event in events:
    for index,detector in enumerate(detectors):
        for cluster in event.clusters:
            if cluster.detector == detector:
                hClusSize[index].Fill(cluster.size)

#Draw histogram
for index,hist in enumerate(hClusSize):
    hist.Scale(1/hist.Integral())
    hist.SetMaximum(0.2)
    hist.Draw("SAME HIST")
    hist.SetLineColor(index+1)
    hist.SetLineWidth(2)
    hist.SetXTitle("Cluster size [pixels]")
    hist.SetYTitle("Counts (normalized to integral)")
    legend.AddEntry(hist,detectors[index])

legend.Draw() 
canvas.Draw()

In [8]:
%jsroot on
path = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456114514_221112120005.pkl" 
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
colors = [30,46,23,13,40]
canvas = ROOT.TCanvas("","",1000,1000)
legend = ROOT.TLegend()

# Calculate histogram
hClusSize = [ROOT.TH1F("Cluster_Size", "Cluster sizes", 41, -0.5, 40.5) for det in detectors]
for event in readEvents(path,1e5):
    for index,detector in enumerate(detectors):
        for cluster in event.clusters:
            if cluster.detector == detector:
                hClusSize[index].Fill(cluster.size)


#Draw histogram
colors = [1,2,3,4,6,7,8,9]

for index,hist in enumerate(hClusSize):
    hist.Scale(1/hist.Integral())
    hist.SetMaximum(0.2)
    hist.Draw("SAME P L")
    hist.SetLineColor(colors[index])
    hist.SetMarkerColor(colors[index])
    hist.SetMarkerStyle(20)   
    hist.SetLineWidth(3)
    hist.SetXTitle("Cluster size [pixels]")
    hist.SetYTitle("Counts (normalized to integral)")
    legend.AddEntry(hist,detectors[index])

legend.Draw() 
canvas.Draw()

Going to process 100000.0 events.


Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).


In [5]:
# %jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
colors = [30,46,23,13,40]
canvas = ROOT.TCanvas("","",1000,1000)
legend = ROOT.TLegend()

# Calculate histogram
hClusSize = ROOT.TH1F("ALPIDE_occupancy", "ALPIDE cluster occupancy", 5, -0.5, 4.5)
for event in readEvents(path,1e7):
    for cluster in event.clusters:
        hClusSize.Fill(cluster.detector,1)

#Draw histogram

hClusSize.Draw("SAME HIST")
hClusSize.SetLineColor(ROOT.kBlack)
#hClusSize.SetGrid(1)
hClusSize.SetLineWidth(2)
hClusSize.SetXTitle("")
hClusSize.SetYTitle("Counts")
hClusSize.LabelsOption("a","X")
legend.AddEntry(hClusSize,detectors[index])
ROOT.gPad.SetGrid()
#legend.Draw() 
canvas.Draw()



Going to process 10000000.0 events.


In [6]:
%jsroot on
canvas = ROOT.TCanvas()
canvas.SetCanvasSize(900,600)

# Calculate histogram
h2DClusSizeVsColWidth = [ROOT.TH2F("Cluster_Size_vs_ColWidth"+det, "Size of cluster vs column width- "+det, 30,0,30,10,0,10) for det in detectors]
for event in events:
    for index,detector in enumerate(detectors):
        for cluster in event.clusters:
            if cluster.detector == detector:
                h2DClusSizeVsColWidth[index].Fill(cluster.size,cluster.colWidth)

#Draw histogram
canvas.Divide(3,2)
for pad in range(5):
    canvas.cd(pad+1)
    h2DClusSizeVsColWidth[pad].Draw("COLZ")
    h2DClusSizeVsColWidth[pad].SetTitle("ALPIDE_"+str(pad)+"; Cluster Size [pixel]; Cluster column width [pixel]")

canvas.Draw()

### Number of clusters

In [7]:
%jsroot on
canvas = ROOT.TCanvas()
canvas.SetCanvasSize(900,900)
canvas.Divide(2,2)
cuts = {0:0,
        1:5,
        2:10,
        3:15
       }
hNClusDet = []
for pad in range(4):
    # Fill histograms
    hNClusDet.append([ROOT.TH1F("nClusters_"+det+"_cut_"+str(pad), "nClusters of events - "+det, 30, 0, 30) for det in detectors])
    for event in events:
        for index,detector in enumerate(detectors):
            count = 0
            for cluster in event.clusters:
                if cluster.detector == detector and cluster.size > cuts.get(pad):
                    count += 1
            if count:
                hNClusDet[pad][index].Fill(count)
                
    # Draw histograms
    canvas.cd(pad+1)
    legend = ROOT.TLegend()
    #hNClus.Draw()
    for index,hist in enumerate(hNClusDet[pad]):
        hist.Scale(1/hist.Integral())
        #hist.SetMaximum(0.15)
        hist.SetLineColor(index+1)
        hist.SetLineWidth(2)
        hist.SetXTitle("# of clusters per event")
        hist.SetYTitle("Counts (normalized to integral)")
        hist.SetTitle("Cluster size > "+str(cuts.get(pad)))
        legend.AddEntry(hist,detectors[index])
        hist.Draw("SAME HIST")
    legend.Draw()
canvas.Draw()

### Correlation plots
+ To-Do make sure this is the right way to do it

In [8]:
nBins = {"x-Axis" : 60, "y-Axis" : 30}
nRange = {"x-Axis" : 16, "y-Axis" : 8}
axis = {"x-Axis":0, "y-Axis":1}

In [9]:
%jsroot on
canvas = ROOT.TCanvas()
legend = ROOT.TLegend()

detector1 = "ALPIDE_3"
detector2 = "ALPIDE_4"
axis1="x-Axis"
axis2="x-Axis"

# Calculate histogram
hCorr = ROOT.TH2F("Corr_"+detector1+"_"+axis1+"_"+detector2+"_"+axis2,"Correlation", nBins.get(axis1),-nRange.get(axis1),nRange.get(axis1),nBins.get(axis2),-nRange.get(axis2),nRange.get(axis2))
hCorr.SetXTitle(detector1 + " - " + axis1)
hCorr.SetYTitle(detector2 + " - " + axis2)
for event in events:
    clustersDetector1 = event.selectDetector(detector1)
    clustersDetector2 = event.selectDetector(detector2)
    #if len(clustersDetector1) == 1 and len(clustersDetector2) == 1:
    if True:
        for cluster1 in clustersDetector1:
            if cluster1.size > 2:
                for cluster2 in clustersDetector2:
                    if cluster2.size > 2:
                        hCorr.Fill(cluster1.localPos[axis.get(axis1)],cluster2.localPos[axis.get(axis2)])
                    
#Draw histogram
hCorr.Draw("COLZ")
canvas.Draw()

In [10]:
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

In [11]:
save_object(events,"myEvents.pkl")

In [12]:
# Dictionaries for axes
nBins = {"x-Axis" : 120, "y-Axis" : 60}
nRange = {"x-Axis" : 15, "y-Axis" : 7.5}
axis = {"x-Axis":0, "y-Axis":1}

In [13]:
def correlationHist(detector1="ALPIDE_1", detector2="ALPIDE_1", axis1="x-Axis", axis2="x-Axis"):
    global nBins,nRange,axName
    hCorr = ROOT.TH2F("Corr"+detector1+axis1+detector2+axis2,"Correlation", nBins.get(axis1),-nRange.get(axis1),nRange.get(axis1),nBins.get(axis2),-nRange.get(axis2),nRange.get(axis2))
    hCorr.SetXTitle(detector1 + " - " + axis1)
    hCorr.SetYTitle(detector2 + " - " + axis2)
    for event in events:
        clustersDetector1 = event.selectDetector(detector1)
        clustersDetector2 = event.selectDetector(detector2)
        for cluster1 in clustersDetector1:
            if cluster1.size > 15:
                for cluster2 in clustersDetector2:
                    if cluster2.size > 15:
                        hCorr.Fill(cluster1.localPos[axis.get(axis1)],cluster2.localPos[axis.get(axis2)])
    return hCorr

In [14]:
%jsroot on
axis1="x-Axis"
axis2="x-Axis"
canvas = ROOT.TCanvas()
hist = correlationHist(detector1="ALPIDE_1", detector2="ALPIDE_1", axis1="x-Axis", axis2="x-Axis")
hist.Draw("COLZ")
canvas.Draw()


In [15]:
%jsroot on
canvas = ROOT.TCanvas()
legend = ROOT.TLegend()

# Calculate histogram
hMyHist = ROOT.TH1F("myHist", "myHist", 30, 0, 30)
for event in events:
    for cluster in event.clusters:
        if int(cluster.detector[-1]) == 1 :
            hMyHist.Fill(cluster.size)

#Draw histogram
hMyHist.Draw()

#legend.Draw() 
canvas.Draw()